This module will demonstrate a basic sizing algorithm used in a ladder-based quoting system. 

Fist we will need to define our configurations. Including the reserve fraction, equity quote (total equity in Quote Units
), and ladder weights. 

In [3]:
# Configurations
reserve_fraction = 0.3
equity_quote = 100
ladder_weights = [0.5, 0.3, 0.2]

Next we will divide the total equity into a reserve and a usable amount.

In [4]:
reserve_quote = equity_quote * reserve_fraction
usable_quote = equity_quote - reserve_quote

print("Equity Quote: ", equity_quote)
print("Reserve Quote: ", reserve_quote)
print("Usable Quote: ", usable_quote)



Equity Quote:  100
Reserve Quote:  30.0
Usable Quote:  70.0


Next we will begin with an even bid and ask size. We will simply divide the usable quote by 2, to gain an even split. 

In [5]:
bid_budget_quote = usable_quote * 0.5
ask_budget_quote = usable_quote * 0.5

print("Bid Budget Quote: ", bid_budget_quote)
print("Ask Budget Quote: ", ask_budget_quote)

Bid Budget Quote:  35.0
Ask Budget Quote:  35.0


From here, we will use our current inventory to adjust the size of the bid and ask quotes. We will use a value from [-1, 1] to represent our current inventory, and more specifically, our relationship with the base unit. For example in the market EURQ-USD, a value of 0.5 would represent us being long the base unit (EURQ). A negative value like -0.5 would represent us being short the base unit (EURQ). If we were long, that indicates that we have tooo much of the base and we should be pushing to sell off some of the excess. If we were short, that indicates that we have tooo little of the base and we should be pushing to buy more of the base. This is the core idea behind the sizing algorithm, the goal being simple, to keep our inventory close to neutral. 

In [8]:
# We will be using 0.5 for this example, but inventory_norm would be imported.
# k, is the sensitivity constant, to indicate how big of an effect inventory has on pricing. 

inventory_norm = 0.5 # Indicates that we are long EURQ, so we would have excess EURQ.

k = 0.6 # Sensitivity constant, to indicate how big of an effect inventory has on pricing. 

m_bid = 1 - k * inventory_norm
m_ask = 1 + k * inventory_norm

print("inventory_norm:", inventory_norm)
print("sensitivity constant:", k)
print("m_bid: ", m_bid)
print("m_ask: ", m_ask)

inventory_norm: 0.5
sensitivity constant: 0.6
m_bid:  0.7
m_ask:  1.3


Now given these new splits, we will use them to manipulate the bid and ask capacity. Note, bid_raw and ask_raw will not always add to the usable_quote, so normalization is required. On the other hand, bid_budget and ask_budget will always add to the usable_quote. In odd cases, where we are adding constraints or limiting the size of either side, normalization may be required. 

In [10]:
bid_raw = bid_budget_quote * m_bid
ask_raw = ask_budget_quote * m_ask

print("Bid Raw: ", bid_raw)
print("Ask Raw: ", ask_raw)

total_raw = bid_raw + ask_raw

# In this example case, the bid_raw and ask_raw do sum to the usable_quote, but they
# do not always. 
print("Total Raw: ", total_raw)

bid_budget = usable_quote * (bid_raw / total_raw)
ask_budget = usable_quote * (ask_raw / total_raw)

print("Bid Budget: ", bid_budget)
print("Ask Budget: ", ask_budget)
print("Usable Quote: ", usable_quote)

Bid Raw:  24.5
Ask Raw:  45.5
Total Raw:  70.0
Bid Budget:  24.5
Ask Budget:  45.5
Usable Quote:  70.0


Lets add an example here where the bid_raw and ask_raw don't sum to the usable_quote.

In [15]:
# Example where the raw bid and ask doesn't equal the usable_quote



Finally from here, we have a bid budger and ask budget, which demonstrate the total size of the bids and asks. We will now divide those into individual ladder levels. For example, in the event that we have 3 bids and asks going simultaneously, we will divide the bid budget and ask budget into 3 parts, and assign each a value dependent on the ladder weighttings. We will likely assign the tightest bid and ask the highest percentage of the budget, and the widest bid and ask the lowest percentage of the budget, in descending order. 

In [16]:
# Recall the ladder weights: [0.5, 0.3, 0.2]
b1 = bid_budget * ladder_weights[0]
b2 = bid_budget * ladder_weights[1]
b3 = bid_budget * ladder_weights[2]

a1 = ask_budget * ladder_weights[0]
a2 = ask_budget * ladder_weights[1]
a3 = ask_budget * ladder_weights[2]

print("bid1 and ask1: ", b1, a1)
print("bid2 and ask2: ", b2, a2)
print("bid3 and ask3: ", b3, a3)

print("Sum of bids: ", b1 + b2 + b3)
print("Sum of asks: ", a1 + a2 + a3)

print("Sum of bids and asks: ", b1 + b2 + b3 + a1 + a2 + a3)

bid1 and ask1:  12.25 22.75
bid2 and ask2:  7.35 13.65
bid3 and ask3:  4.9 9.1
Sum of bids:  24.5
Sum of asks:  45.5
Sum of bids and asks:  70.0


Note, this would be fine working with pricer. As we would quote sizes for different ladder. We could recall pricer with a thinner orderbook to force price selection for next ladder step.